lamUを0にすれば softmargin SVMである．

実験1と割合（サンプル数）は同様だが，抽出元が異なる．ラベル無しデータは無い．
- ラベル付きデータの割合 p = 0.1*(0.1〜0.8) , D_l selected from D_0 (predefined training data: 10% of anotated)
- テストデータの割合 r = 0.9 , \Omega - D_0

In [ ]:
from indianpines.dataset import load as load_pines
from s3vm_pines.module import recategorize_csv, train_test_split, colored_map, labeled_unlabeled_test_split

pines = load_pines(pca=20,recategorize_rule=recategorize_csv, include_background=True,gt_gic=False)

In [ ]:
prop_train = 1348/10320
print(prop_train)

status, status_name = train_test_split(prop_train=prop_train, recategorize_rule=recategorize_csv, gt_gic=False)

for i, s in enumerate(status_name):
    n = status[status==i].shape[0]
    print(f'{i}:{s}:{n}')

## ラベル付きトレーニングデータの割合に対する評価


In [ ]:
#| label: fig_test_with_various_labeled
#| fig-cap: "Testing with Various Labeled Data Proportion"
#| output: false

from qns3vm import QN_S3VM_OVR
from qns3vm.tools import classification_error
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import random
from matplotlib import style
style.use(['science','ieee'])


rg = random.Random()

seed_l = 83988848
seed_u = 83988848

prop_train_l_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

acc = list()
err = list()
f1 = list()
acc_svm = list()
err_svm = list()
f1_svm = list()
fig2 = plt.figure()
ax21 = [0 for _ in prop_train_l_list]
for i, prop_train_l in enumerate(prop_train_l_list):
    l_u_t_status, l_u_t_status_name = labeled_unlabeled_test_split(prop_train_l=prop_train_l,status=status,features=pines.features, target=pines.target, cordinates=pines.cordinates, unlabeled_type = 'from_train',seed_l=seed_l, seed_u=seed_u)

    for i, s in enumerate(l_u_t_status_name):
        n = l_u_t_status[l_u_t_status==i].shape[0]
        print(f'{i}:{s}:{n}')

    X_train_l = pines.features[l_u_t_status==3].tolist()
    L_train_l = pines.target[l_u_t_status==3].tolist()
    X_train_u = pines.features[l_u_t_status==2].tolist()
    X_test = pines.features[l_u_t_status==1].tolist()
    L_test = pines.target[l_u_t_status==1].tolist()

    ax21[i] = fig2.add_subplot(3,3,i)
    colored_map(ax21[i], L_train_l, pines.cordinates[l_u_t_status==3], recategorize_rule=recategorize_csv, gt_gic=False)
    ax21[i].set_title(f'{prop_train_l =}')

    clf = QN_S3VM_OVR(X_train_l, L_train_l, X_train_u, random_generator=rg, lam=0.008765625, lamU=0, sigma=0.5, kernel_type="RBF", estimate_r=0.0)
    clf.train()
    preds = clf.predict(X_test)
    err_ = classification_error(preds,L_test)
    acc_ = accuracy_score(L_test, preds)
    f1_ = f1_score(L_test, preds, average="micro")
    err.append(err_)
    acc.append(acc_)
    f1.append(f1_)

    clf_svm = QN_S3VM_OVR(X_train_l, L_train_l, X_train_u, random_generator=rg, lam=0.008765625, lamU=0.8, sigma=0.5, kernel_type="RBF", estimate_r=0.0)
    clf_svm.train()
    preds_svm = clf_svm.predict(X_test)
    err_svm_ = classification_error(preds_svm,L_test)
    acc_svm_ = accuracy_score(L_test, preds_svm)
    f1_svm_ = f1_score(L_test, preds_svm, average="micro")
    err_svm.append(err_svm_)
    acc_svm.append(acc_svm_)
    f1_svm.append(f1_svm_)


ax22 = fig2.add_subplot(3,3,9)
ax22.plot(prop_train_l_list, acc, label='S3VM')
ax22.plot(prop_train_l_list, acc_svm, label='SVM')
ax22.set_xlabel(r'Amount of Labeled Data [$\%$]')
ax22.set_ylabel(r'Accuracy Score[$\%$]')
ax22.set_title('Testing with Various Labeled Data Proportion')